__Cuaderno de trabajo de:__ Nombre Apellido

# Regresión lineal simple

Vamos a seguir el capítulo 3 del libro ["Introduction to Statistical Learning"](http://www-bcf.usc.edu/~gareth/ISL/), una elección popular escrita por protagonistas del campo, ameno de leer y que podéis descargar gratis de su página web. <br>
El libro usa R, pero los ejemplos han sido [migrados a python](https://github.com/JWarmenhoven/ISLR-python).
En este cuaderno sólo vamos a trabajar el caso más sencillo: la "Regresión lineal simple".<br>  
- Cargar los datos
- Análisis de regresión
- Regresión Lineal Simple
    - numpy.polyfit
    - scikit_learn
    - statsmodels

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.linear_model as skl_lm
import statsmodels.formula.api as smf

%matplotlib inline
plt.style.use('seaborn-white')

## Cargamos los datos

Cargamos "advertising.cvs", es un conjunto de datos publicitarios, las ventas, en miles
de unidades, en función de los presupuestos de TV, radio y periódicos, en miles de
dólares, para 200 mercados diferentes. 

In [ ]:
advertising = pd.read_csv('advertising.csv', usecols=[1,2,3,4])
advertising.info()

In [ ]:
advertising.head()

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize = (12, 3))
sns.scatterplot(x='TV', y='Sales',data=advertising,ax=ax1)
sns.scatterplot(x='Radio', y='Sales',data=advertising,ax=ax2)
sns.scatterplot(x='Newspaper', y='Sales',data=advertising,ax=ax3)
plt.show()


## Regresión y ajuste de modelos


El análisis de regresión consiste en encontrar un  **modelo** que relaciona los valores medidos de una variable **objetivo** (tb se llama la **respuesta**) en función de un conjunto de variables **explicativas** (tb **variables predictoras**, o **regresores**).

Los valores medidos en el mundo real nunca se ajustan de forma perfecta a un modelo, debido en primer lugar a errores de medida, pero también a que cualquier modelo matemático es una *simplificación* del mundo real, y si tuviera en cuenta todos los factores que influyen en un conjunto de variables, sería inmanejable.

Por tanto, no tiene sentido aspirar a encontrar un modelo que prediga exactamente los valores medidos, y debemos admitir que el modelo cometerá un cierto error.

Un modelo útil encuentra una relación funcional sencilla en conjuntos de pocas variables. Se trata de explicar una variable objetivo en función de otro conjunto de variables mejor conocidas o más fáciles de medir. El  **análisis de regresión**  (más exactamente, el análisis de regresión  *paramétrico*) permite encontrar un modelo explicativo en dos etapas:


 1. Nuestro conocimiento del tema en cuestión nos permite escribir un modelo que afirma que la variable  $Y$  es una función de las variables $X_1,\dots,X_p$ que podemos conocer, y de otras variables $\varepsilon_1,\dots,\varepsilon_k$ que no conocemos. La variable  $Y$ se suele llamar la **respuesta** y las variables  $X_1,\dots,X_p$ se llaman  **variables predictoras**:

$$
Y = f(X_1,\dots,X_p, \varepsilon_1,\dots,\varepsilon_k)
$$
 
 Sin embargo, no conocemos a priori la expresión de la función. En la **regresión paramétrica**, la función depende de unos pocos  **parámetros**  libres.


## Regresión Lineal Simple

Ajustamos un modelo lineal entre dos variables:

$$
Ventas = \beta_0 + \beta_1 \times TV + \varepsilon \thickapprox \beta_0 + \beta_1 \times TV
$$

Cada par de valores $(\beta_0, \beta_1)$ corresponde a una recta distinta.

### Ajuste por minímos cuadrados
Como podemos ver nuestra recta de ajuste quedará definida cuando obtengamos $\beta_0$ y $\beta_1$, es decir tenemos los siguientes datos:

Dibujamos la recta ajustada por mínimos cuadrados:
$$(x_1,y_1),(x_2,y_2),...,(x_n,y_n)$$
El residuo en cada punto $e_i$ representaría la distancia del punto a la recta estimadada 
$$e_i=y_i-\beta_0 - \beta_1 \times x_i$$
Donde la suma de los residuos $RSS$ quedaría como:
$$ RSS=e^2_1+e^2_2+...+e^2_n = (y_1-\beta_0 - \beta_1 \times x_1)^2+...+(y_n-\beta_0 - \beta_1 \times x_n)^2$$
Si minimizamos $RSS$ obtenemos que:
$$\beta_1 = \frac{\sum_{i=1}^{n}(x_i-\overline{x})(y_i-\overline{y})}{\sum_{i=1}^{n}(x_i-\overline{x})^2}
\newline
\beta_0 = \overline{y}-\beta_1 \overline{x} $$
donde $\overline{x}$ representa la media $\overline{x}=\frac{1}{n}\sum_{i=1}^{n}x_i$

In [ ]:
def ajuste_minimos_cuadrados(x,y):
    b1=0
    b1_n=0
    b1_d=0
    for i in range(len(x)):
        b1_n = b1_n + (x[i]- np.mean(x))*(y[i]- np.mean(y))
        b1_d = b1_d + (x[i]-np.mean(x))**2
    b1=b1_n/b1_d
    b0=np.mean(y)-b1*np.mean(x)
    return b1,b0


x = np.array([1.0,2.0,3.0,4.0,5.0,6.0])
y = np.array([1.2,1.5,2.4,3.1,5.6,5.8])
#pintamos los puntos
plt.plot(x, y, 'o',color="cornflowerblue")

b1,b0 = ajuste_minimos_cuadrados(x,y)
#pintamos la recta
plt.plot(x, b1*x + b0,color='orange')

#pintamos los residuos
for i in range(len(x)):
    aux_x=np.array([x[i],x[i]])
    aux_y=np.array([y[i],b1*x[i]+(b0)])
    plt.plot(aux_x,aux_y,color="r")

    
plt.show()
print("y = (",b1,')* x +(',b0,')')

In [ ]:
#Cargamos los datos definidos en advertising.csv
x = np.array(advertising.TV) #np.array(advertising['TV'])
y = np.array(advertising.Sales)
plt.plot(x, y, 'o',color="cornflowerblue", markersize=3)
b1,b0 = ajuste_minimos_cuadrados(x,y)
plt.plot(x, b1*x + b0,color='orange')
for i in range(len(x)):
    aux_x=np.array([x[i],x[i]])
    aux_y=np.array([y[i],b1*x[i]+(b0)])
    plt.plot(aux_x,aux_y,color="r",linewidth=0.5)

    
plt.show()
print("y = (",b1,')* x +(',b0,')')

In [ ]:
x = np.array(advertising.TV) #np.array(advertising['TV'])
y = np.array(advertising.Sales)
plt.plot(x, y, 'o',color="cornflowerblue")
b1, b0 = np.polyfit(x, y, 1) #ajustamos a polinomio de orden 1
plt.plot(x, b1*x + b0,color='orange')
print("y = (",b1,')* x +(',b0,')')

#m=[]
#m=np.polyfit(x, y, 1)
#print(m)
#plt.plot(x, m[0]*x + m[1])

plt.show()

In [ ]:
#seaborn
sns.regplot(x=advertising.TV, y=advertising.Sales, order=1, ci=None, scatter_kws={'color':'cornflowerblue', 's':30},color='orange')
# ajusta aun polinómio de orden = order, para ello usa numpy.polyfit
plt.xlim(-10,310)
plt.ylim(ymin=0)
plt.show()

### scikit_learn

La librería ``scikit_learn`` es una elección popular que cubre muchos algoritmos de estadística y machine learning.
El funcionamiento de la regresión lineal es similar al de otros modelos más sofisticados:

  - 1: Definimos un objeto de tipo "LinearRegression"

```python
regr = skl_lm.LinearRegression()
```

  - 2: El método ``fit`` **"ajusta"** la recta, encontrando los valores de $(a,b)$ para los que el error cuadrático cometido es menor. Necesita dos argumentos:
      - Un **DataFrame** X con las variables predictoras. No acepta una serie, si sólo tenemos una variable predictora usamos un DataFrame con una sóla columna.
      - Una **Serie** y con la variable objetivo.

También admite arrays de numpy.

```python
X = advertising[['TV']]
y = advertising.Sales

regr.fit(X,y)
```

y ya tenemos el objeto ``regr`` que contiene la recta ajustada por mínimos cuadrados.

In [ ]:
# Usamos la librería scikit_learn
regr = skl_lm.LinearRegression()

X = advertising[['TV']]
y = advertising.Sales

regr.fit(X,y)

#El objeto regr contiene los valores de los parametros
#a,b donde y = b1*x + b0
print("y = ",regr.coef_[0],'* x +',regr.intercept_)

### Aplicar el modelo

Una vez hemos encontrado los valores de los coeficientes que mejor ajustan a los datos, podemos usar esos valores para predecir el nivel de ventas, para la inversión en TV que se ha planificado para el próximo periodo.

$$
Ventas\approx \beta_0 + \beta_1 \times TV
$$

Si usamos el modelo para predecir la respuesta ante valores de las variables predictoras que están fuera del rango de los datos de **entrenamiento** (los datos que usamos para hacer el ajuste), decimos que estamos **extrapolando**.

Con scikit-learn:

  - Podemos leer los coeficientes $(\beta_0, \beta_1)$

```python
print(regr.intercept_)  #beta0
print(regr.coef_)       #beta1
```

  - Podemos usar el modelo para predecir qué ocurrirá ante determinados valores de las variables regresoras.

```python
regr.predict([[200]])
```

In [ ]:
#¿Qué ventas esperamos si la inversión en TV es de 200?
print(regr.predict([[200]]))

#¿Podemos pasar varios valores a la vez, pero tiene que
#ser un array con shape L x 1
xs = np.array([[200], [300]])
print(regr.predict(xs))

In [ ]:
# Suele ser útil introducir los datos como un DataFrame
X = pd.DataFrame({
    'TV': [200,300]
})

regr.predict(X)

In [ ]:
#Otra notación alternativa para definir un DataFrame
X = pd.DataFrame(
    [
        [200], [300]    
    ],
    columns=['TV']
)

regr.predict(X)

In [ ]:
plt.figure(figsize=(7,7))
sns.regplot(x=advertising.TV, y=advertising.Sales, order=1, ci=None,  scatter_kws={'color':'cornflowerblue', 's':30})
xs = np.array([[350], [400]])
plt.scatter(xs, regr.predict(xs), color='purple', label='extrapolando')
plt.xlim(-10,410)
plt.ylim(ymin=0)
plt.legend(loc='upper left')
plt.show()

In [ ]:
X = advertising[['TV']]
y = advertising.Sales

regr.score(X,y) #R²

## ¿Cómo decidimos si un modelo es bueno?

Supongamos que el mejor modelo lineal que hemos encontrado es $y=f(x)=a * x + b $:

 - **TSS**: Total sum of squares: $\Sigma_i (y_i-\bar{y})^2$ (sumamos el cuadrado de la diferencia entre el dato $y_i$ y la media $\bar{y}$)
 - **RSS**: Residual sum of squares:  $\Sigma_i (y_i-f(x_i))^2$ (sumamos el cuadrado de la diferencia entre el dato $y_i$ y la predicción $f(x_i)$ usando nuestro modelo)
 - **RSS/TSS**: cociente entre la "varianza residual" y la "varianza total"
 - **R-cuadrado**: "porcentaje de la varianza que el modelo explica". 
$$
R^2 = 1 - \frac{RSS}{TSS}
$$

```python
#Calcula R^2
regr.score(X,y)
```

El **"estadístico" $R^2$** sólo puede tomar valores entre 0 y 1:
   - Si vale 0, es que RSS=TSS, luego el modelo tiene el mismo error cuadrático total que tomar $f(x)=\bar{y}$.
   - Si vale 1, es que RSS=0, luego los datos se ajustan de forma "perfecta" al modelo.

> - *¿Cuándo ocurre RSS=1?*

Cuando $y_i=f(x_i) = \beta_0 + x_i\beta_1$ para cada $i$. Es decir, *cuando hay una relación lineal entre el vector $x_i$ y el vector $y_i$*. Pero sabemos que hay una relación lineal entre dos series de puntos si y sólo la correlación es 1 o -1. De hecho, *para regresión lineal simple*, $R^2$ es *exactamente* la correlación al cuadrado.

Sin embargo, veremos que para regresión lineal múltiple, el estadístico $R^2$ ofrece información que las correlaciones no aportan.

### Statsmodels 

Vamos a usar también la librería statsmodel, que imita la sintaxis de la regresión lineal en ``R``.

La fórmula ``Sales ~ TV`` significa que busca la recta de regresión
$$
Sales\approx \beta_0 + \beta_1\times TV + \varepsilon
$$

 - Ajustamos el modelo $Sales\approx \beta_0 + \beta_1\times TV + \varepsilon$ usando el DataFrame ``advertising``.
 
```python
recta = smf.ols('Sales ~ TV', advertising).fit()
```

In [ ]:
#Ajustamos el modelo $Sales\approx a *TV + b +\varepsilon$ usando el DataFrame ``advertising``
recta = smf.ols('Sales ~ TV', advertising).fit()

 - Para predecir las ventas para uno o varios valores de inversión en TV, es necesario preparar un DataFrame que tenga una serie de datos de nombre 'TV':

```python
nuevos_datos = pd.DataFrame({
    'TV': [TV_1, ..., TV_n ]
})
recta.predict(nuevos_datos)
```

In [ ]:
nuevos_datos = pd.DataFrame({
    'TV': [100,150,200,250,300,350]
})
recta.predict(nuevos_datos)

In [ ]:
recta.params

In [ ]:
recta.rsquared

#### Información extra sobre los coeficientes

stasmodels ofrece información extra sobre el resultado de la regresión.

In [ ]:
recta.summary().tables[1]

La primera fila contiene información sobre $a$ (**intercept**) y la segunda sobre $b$:
 - **coef** es la estimación del parámetro por mínimos cuadrados
 - **std err** es una medida de la incertidumbre en la estimación del parámetro
 - Las columnas **[0.025** y **0.975]** son los extremos de un "intervalo de confianza al 95%" para el valor del parámetro.
 - **t** es el valor del "t-statistic": si es bajo, quiere decir que la variable TV realmente no es de gran ayuda para predecir las ventas.
 - **P>|t|** es el *"p-valor"* asociado al valor anterior del t-estadístico. Toma valores entre 0 y 1. Un valor cercano a 1 se interpreta como *"se puede defender que Sales no depende de TV"*, y un valor cercano a 0 viene a ser *"realmente parece que TV es un buen predictor del valor de Sales"*. Un valor típico para comparar es p=0.05 o p=0.01. En este caso es aún menor.

Por favor leed las lecturas recomendadas (por ejemplo, la sección 3.1 del Introduction to Statistical Learning) para profundizar sobre estos números. La semana que viene hablaremos de ellos más despacio.

<hr>
<b><font color='red'>Ejercicio 01</font></b>  

Hemos trabajado los datos para la TV, ajusta ahora los datos para la radio y los periodicos, pinta los puntos y la recta.
Haz que salgan tambien los diferentes parametros de ajuste, es decir el valor de a, b y R²

In [ ]:
#para la radio
"""Esribe tu código aquí"""

In [ ]:
#para los periódicos
"""Esribe tu código aquí"""

<hr>
<b><font color='red'>Ejercicio 02</font></b>  

Tratamos de predecir la temperatura a la que hierve el agua ( T ), conocida la presión atmosférica ( P ) en el lugar y momento en que hacemos el experimento.

Para ello, contamos con un conjunto de mediciones de ambas variables, con la temperatura en grados Fahrenheit y la presión en pulgadas de mercurio (sea lo que sea, es una unidad de medidad de presión). Por ejemplo, en un cierto punto de los Alpes, un cierto día, el barómetro marcaba 20.79 pulgadas de mercurio, y el agua hirvió a 194.5 grados Fahrenheit. Las mediciones se realizaron en el mismo lugar geográfico, pero en días distintos, con distintas condiciones atmosféricas y quizá incluso por personas distintas. En estas condiciones, es imposible que ningún modelo prediga con exactitud el valor de T en función de P, pero esperamos que lo haga con un margen de error moderado.

$$
\begin{array}{rrrrrrrrrr}
T&194.5 & 194.3 & 197.9 & 198.4 & 199.4 & 199.9 & 200.9 & 201.1 & 201.4 & 201.3 & 203.6 & 204.6 & 209.5 & 208.6 & 210.7 & 211.9 & 212.2\\
P &20.79&  20.79&  22.4 &  22.67&  23.15&  23.35&  23.89&  23.99& 24.02&  24.01&  25.14&  26.57&  28.49&  27.76&  29.04&  29.88& 30.06
\end{array}
$$

 - Ajusta un modelo lineal a estos datos
 - Calcula el valor de $R^2$ del ajuste
 - Usa el modelo para predecir la temperatura de ebullición del agua cuando la presión es de 26 "pulgadas de mercurio".

In [ ]:
datos = np.array(
        [(20.79,194.50),(20.79,194.30),(22.40,197.90),(22.67,198.40),
         (23.15,199.40),(23.35,199.90),(23.89,200.90),(23.99,201.10),
         (24.02,201.40),(24.01,201.30),(25.14,203.60),(26.57,204.60),
         (28.49,209.50),(27.76,208.60),(29.04,210.70),(29.88,211.90),
         (30.06,212.20)])

In [ ]:
"""Esribe tu código aquí"""

<hr>
<b><font color='red'>Ejercicio 03</font></b>  

In [ ]:
#Utliza el archivo super.cvs
#Haz un ajuste por mínimos cuadrados del número de articulos respecto al precio
"""Esribe tu código aquí"""

In [ ]:
#Utliza el archivo super.cvs
#Haz un ajuste por mínimos cuadrados del número de articulos respecto al precio
"""Esribe tu código aquí"""

In [ ]:
#¿En el caso de que una persona comprase 40 artículos cual sereía la estimación del precio utilizando el valor medio y el modelo que has utilizado ?
"""Esribe tu código aquí"""